# The sampler version of VQE

The VQE is originally formulated for minimizing the expectation value of some Hamiltonian $H$. The best solution is the parameter set that yields the smallest energy. 

In a classical optimization setting, where $H$ is a diagonal matrix, the goals, results and methods are slightly different than in the general case:
* the best result is the best _measurement/bitstring_, not the best energy
* we can use aggregation functions on the single measurements (like CVaR) which is only possible for diagonal Hamiltonians 
* the Hamiltonian can be given as function on bitstrings

Therefore we suggest to add a `SamplerVQE` (`SamplingVQE`?) that's specific to this use case and can be leveraged in QAOA and Qiskit Optimization.

### Signature

In [ ]:
class SamplerVQE:
    def __init__(
        self,
        ansatz: QuantumCircuit,
        optimizer: Union[Optimizer, Minimizer],
        sampler: Sampler,
        gradient: EstimatorGradient,  # bit weird that's it's not a sampler gradient
        aggregation: Union[float, AGGREATOR],  # CVaR alpha if a float, but any aggregation possible
    ) -> None:
        ...
    
    def compute_minimum_eigenvalue(
        self,
        operator: OperatorBase,  # could also support a function f: bitstr -> float
        aux_operators: ListOrDict[operator]
    ) -> SamplerVQEResult:
        ...

In [ ]:
@dataclass
def SamplerVQEResult:
    # same as VQE
    eigenvalue: float
    cost_function_evals: int
    optimal_point: np.ndarray
    optimal_parameters: Dict[Parameter, float]
    optimizer_time: float
    
    # different
    best_measurement: str
    final_samples: QuasiDistribution  # could also be called "eigenstate"

### QAOA

**Note** If QAOA inherits from `SamplerVQE` the new version can only handle diagonal Hamiltonians, not like right now any Hamiltonian!

#### Option 1

QAOA inherits from `SamplerVQE` and can from now on only handle diagonal Hamiltonians.

In [ ]:
class QAOA(SamplerVQE)

#### Option 2

QAOA still inherits from `VQE` and we add a new `SamplerQAOA` inheriting from `SamplerVQE`

In [ ]:
class QAOA(VQE)
class SamplerQAOA(SamplerVQE)

### SamplerMinimumEigensolver

Should we add a `SamplingMinimumEigensolver` as common base class of `SamplerVQE`, `(Sampler)QAOA` and add a `SamplerNumPyMES`?

#### Pro SamplerMES

* The `SamplerVQE` cannot be used as drop-in replacement for other MES as it only supports diagonal Hamiltonians.
* Results types also differ
* We can have a classical, exact reference algo: `SamplerNumPyMES`

#### Contra SamplerMES

The alternative solution would be that `SamplerVQE` & co. derive from `MinimumEigensolver` but raise an error if the Hamiltonian is not diagonal.

* The `MinimumEigenOptimizer` could continue to just accept any MES (otherwise we have to start supporting MES and SamplerMES)
